<a href="https://colab.research.google.com/github/marcory-hub/hailo-colab/blob/main/ptToHef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversion from PyTorch (pt) via HAR to HEF

Goal of this notebook is to make a `HEF` file is the file that runs on the `hailo-8l` device that is on the AI-kit. For a schematic overview and more details check the hailo docs about the [model build process](https://hailo.ai/developer-zone/documentation/dataflow-compiler-v3-29-0/?sp_referrer=overview/overview.html).

Credits to trieut415! A lot of the code from hailo is adjusted inspired on his [post](https://community.hailo.ai/t/guide-to-using-the-dfc-to-convert-a-modified-yolov11-on-google-colab/7131/3) in *the* hailo community. Especially the solution to run the Dataflow Compiler in a virtual environment solved my initial problem. Furthermore, the codeblock to make the calibration data is more robust.

## Overview
### 1. Parsing from .pt to .har:
- Input: yolo.pt file
- Output: har file (model representation and parameters (32-bits weights))

### 2. Model Optimization:
- input: har file (32-bits) and calibration images
- output: har file (optimized model representation and parameters (quantized weights))

  Conversion of the har file with float32 parameters to integers. To convert te parameters tun the model emulation in native mode on a small set of images (not annotated).

  #### Substeps

  1. Prepare callibration set
  2. Load har file (32-bits) from model conversion
  3. Create model script


### 3. Model Compilation:
- input: har (optimized)
- output: hef

  The quantized model is compiled into a specific binary format called HEF (Hailo Executable Format). This format is optimized for the Hailo device's architecture and allows for efficient execution of the model's operations.

## Before your start
1. Make a folder `hailo` in your Google Drive.
2. Download hailo dataflow compiler  from https://hailo.ai/developer-zone/software-downloads/ (you need to make an account) and upload it to your Google Drive. To check the python version of Colab you can run the command below.

In [ ]:
!python --version

3. Collect a set of (at least) 1024 images needed for callibration. These images need no annotation, but should be representatieve. Zip it preferably with the name calibrationDataset.zip (On mac use `ditto -c -k --norsrc --keepParent calibrationDataset calibrationDataset.zip`)
4. Till Step 2.1 free CPU is sufficient, the optimization step load all images at once, then 32GB RAM is needed.

## Install Dataflow Compiler (DFC) in virtual environment (venv)

For the next codeblock, make sure you downloaded hailo dataflow compiler (python 3.10) from https://hailo.ai/developer-zone/software-downloads/ and copied the .whl to your Google Drive. Change the filename if their was an update.


In [1]:
from google.colab import drive

# Mount google drive
drive.mount('/content/gdrive')

# Update and install packages needed for DataFlow Compiler
!sudo apt-get update
!sudo apt-get install -y python3-dev python3-distutils python3-tk libfuse2 graphviz libgraphviz-dev

# Make a virtual environment to install the DFC in
!pip install --upgrade pip virtualenv
!virtualenv my_env

# Installing the Dataflow Compiler, update the filename if needed
!my_env/bin/pip install /content/gdrive/MyDrive/hailo/hailo_dataflow_compiler-3.29.0-py3-none-linux_x86_64.whl

# Check the version and show help information
!my_env/bin/hailo --version

# Uncomment for help information
#!my_env/bin/hailo -h

Mounted at /content/gdrive
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,840 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,633 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:13 http://archive.ubuntu.com/ubuntu

## 1.  Parsing from PyTorch (.pt) file to ONNX file

In [2]:
!pip install ultralytics
!pip install onnx
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.3/904.3 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 112.4 MB/s eta 0:00:00
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 51.9 MB/s eta 0:00:00
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)


In [3]:
import torch

# Set paths
model_path = "/content/gdrive/MyDrive/hailo/best.pt" # @param {type:"string"}

# Load our model into our environment
checkpoint = torch.load(model_path)
model = checkpoint['model']

model = model.float()
model.eval()

# Dummy input in FP32
dummy_input = torch.randn(16, 3, 640, 640, dtype=torch.float) # Adjust if needed

# Export to ONNX
torch.onnx.export(
    model,
    dummy_input,
    "/content/best.onnx",
    export_params=True,
    opset_version=11,  # Adjust opset version if needed
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output']
)
print("ONNX model exported successfully!")

<ipython-input-3-5a7f1978295e>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/modules/head.py:105: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.format != "imx" and (self.dynamic or self.shape != shape):
/usr/local/lib/python3.10/dist-packages/ultralytics/utils/tal.py:338: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, stride in enumerate(strides):


ONNX model exported successfully!


In [4]:
import onnx
import onnxruntime as ort
import torch

# Set path to ONNX model
onnx_model_path = "/content/best.onnx" # @param {type:"string"}

# Load the ONNX model
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")

# Test the ONNX model with ONNX Runtime
dummy_input = torch.randn(16, 3, 640, 640).numpy()
ort_session = ort.InferenceSession(onnx_model_path)
outputs = ort_session.run(None, {"input": dummy_input})
print(outputs[0])

ONNX model is valid!
[[[     10.026      15.618      17.524 ...      536.02      572.84      586.55]
  [     10.264      13.359      14.438 ...      528.71      529.72       538.7]
  [     19.911       30.98      34.491 ...      207.83       133.7      107.56]
  ...
  [ 2.3544e-05  1.3113e-05  3.9935e-06 ...  5.9605e-08  1.0133e-06  1.6689e-06]
  [  0.0023622    0.005598   0.0046171 ...  1.7881e-07  1.0133e-06  1.6093e-06]
  [ 0.00011188  0.00014335  7.7665e-05 ...           0  2.9802e-08  8.9407e-08]]

 [[     8.0225       13.37      22.991 ...      528.91      562.47      582.91]
  [     11.252      13.527      14.643 ...      533.15      530.06      538.33]
  [     15.814      26.139      43.627 ...      221.69      154.46      114.29]
  ...
  [ 1.3113e-05  7.8678e-06   5.424e-06 ...  1.1921e-07  5.3346e-06   4.679e-06]
  [  0.0009155  0.00094476   0.0020858 ...  2.9802e-08   2.116e-06   2.116e-06]
  [ 4.2051e-05  2.5392e-05  2.5392e-05 ...           0  8.9407e-08  5.9605e-08]]

 [[



1. Select the hardware architecture. For the raspberry AI-kits it is `hailo8l`.
2. Open the [netron](https://netron.app/) site, Click `Open Model` and select your onnx file on your local computer.
3. To identify the end nodes, they are the nodes right before the post-processing operations at the very bottom of the model. Their are 2 end nodes per map. I used a search for `onnx::Reshape` to get to the two `conv` layers that pointed to the `onnx::Reshape`.

  In an unmodified yolo11 model this are the endpoints:
  ```
"/model.23/cv2.2/cv2.2.2/Conv",
"/model.23/cv3.2/cv3.2.2/Conv",
"/model.23/cv2.1/cv2.1.2/Conv",
"/model.23/cv3.1/cv3.1.2/Conv",
"/model.23/cv2.0/cv2.0.2/Conv",
"/model.23/cv3.0/cv3.0.2/Conv",
```
  If they are different, then depicted above you have to change it in the code block below.
4. Check the net_input_shapes in netron. Adjust it if your "input layer name": [batch, rgb, image size] are different from:
  ```
  "images": [1, 3, 640, 640]
  ```

5. Run the codeblocks below. The har file is created by the command `runner.translate_onnx_model` and saved with `runner.save_har`. To use the DFC in the venv we make and save the python code in the first codeblock and run it in the venv in the second codeblock. More details about the conversion can be found in the [Parsing tutorial](https://hailo.ai/developer-zone/documentation/dataflow-compiler-v3-29-0/?sp_referrer=tutorials_notebooks/notebooks/DFC_1_Parsing_Tutorial.html).

(Note: onnx_model_name is without .onnx)

In [5]:
with open("translate_model.py", "w") as f:
    f.write("""

from hailo_sdk_client import ClientRunner

# Set hailo hardware architecture and onnx model and model path
chosen_hw_arch = "hailo8l" # @param ["hailo8l", "hailo8", "hailo8r", "hailo10h", "hailo15h", "hailo15m"]
onnx_model_name = "best" # @param {type:"string"}
onnx_path = "/content/best.onnx" # @param {type:"string"}

print("Starting model translation...")

# Initialize the ClientRunner
runner = ClientRunner(hw_arch=chosen_hw_arch)

# Change the end_node_names if netron show different end nodes
end_node_names = [
  "/model.22/cv2.0/cv2.0.2/Conv",
  "/model.22/cv3.0/cv3.0.2/Conv",
  "/model.22/cv2.1/cv2.1.2/Conv",
  "/model.22/cv3.1/cv3.1.2/Conv",
  "/model.22/cv2.2/cv2.2.2/Conv",
  "/model.22/cv3.2/cv3.2.2/Conv",
]

try:
    # Translate the onnx model to har file
    hn, npz = runner.translate_onnx_model(
        onnx_path,
        onnx_model_name,
        end_node_names=end_node_names,
        net_input_shapes={"input": [16, 3, 640, 640]},  # Adjust input shapes if needed
    )
    print("Model translation successful.")
except Exception as e:
    print(f"Error during model translation: {e}")
    raise

# Save the har file
hailo_model_har_name = f"{onnx_model_name}_hailo_model.har"
try:
    runner.save_har(hailo_model_har_name)
    print(f"HAR file saved as: {hailo_model_har_name}")
except Exception as e:
    print(f"Error saving HAR file: {e}")


""")

In [6]:
# Run model translate model
!my_env/bin/python translate_model.py

Starting model translation...
[info] Translation started on ONNX model best
[info] Restored ONNX model best (completion time: 00:00:00.26)
[info] Extracted ONNXRuntime meta-data for Hailo model (completion time: 00:00:01.41)
[info] NMS structure of yolov8 (or equivalent architecture) was detected.
[info] In order to use HailoRT post-processing capabilities, these end node names should be used: /model.22/cv2.0/cv2.0.2/Conv /model.22/cv3.0/cv3.0.2/Conv /model.22/cv2.1/cv2.1.2/Conv /model.22/cv3.1/cv3.1.2/Conv /model.22/cv3.2/cv3.2.2/Conv /model.22/cv2.2/cv2.2.2/Conv.
[info] Start nodes mapped from original model: 'input': 'best/input_layer1'.
[info] End nodes mapped from original model: '/model.22/cv2.0/cv2.0.2/Conv', '/model.22/cv3.0/cv3.0.2/Conv', '/model.22/cv2.1/cv2.1.2/Conv', '/model.22/cv3.1/cv3.1.2/Conv', '/model.22/cv2.2/cv2.2.2/Conv', '/model.22/cv3.2/cv3.2.2/Conv'.
[info] Translation completed on ONNX model best (completion time: 00:00:09.52)
Model translation successful.
[info

Optional: Save hailo_model.har to google drive.

In [7]:
from google.colab import drive

# Copy best_hailo_model.har to google drive hailo folder
!cp /content/best_hailo_model.har /content/gdrive/MyDrive/hailo/best_hailo_model.har

## 2. Model optimization
The optimization from Hailo replaced by the optimization in the guide from trieut415.

1. Print dictionary of layers and operations
2. Load har
3. create model script


1. Print layers

In [8]:
with open("inspect_layers.py", "w") as f:
    f.write("""

from hailo_sdk_client import ClientRunner

# Load the HAR file
har_path = "/content/best_hailo_model.har" # @param {type:"string"}

runner = ClientRunner(har=har_path)

from pprint import pprint

try:
    # Access the HailoNet as an OrderedDict
    hn_dict = runner.get_hn()  # Or use runner._hn if get_hn() is unavailable
    print("Inspecting layers from HailoNet (OrderedDict):")

    # Pretty-print each layer
    for key, value in hn_dict.items():
        print(f"Key: {key}")
        pprint(value)
        print("\\n" + "="*80 + "\\n")  # Add a separator between layers for clarity

except Exception as e:
    print(f"Error while inspecting hn_dict: {e}")

""")

In [9]:
# Run model inspect layers
!my_env/bin/python inspect_layers.py

Inspecting layers from HailoNet (OrderedDict):
Key: name
'best'


Key: net_params
OrderedDict([('version', '1.0'),
             ('stage', 'HN'),
             ('clusters_placement', [[]]),
             ('clusters_to_skip', []),
             ('output_layers_order',
              ['best/conv41',
               'best/conv42',
               'best/conv52',
               'best/conv53',
               'best/conv62',
               'best/conv63']),
             ('transposed_net', False),
             ('net_scopes', ['best'])])


Key: layers
OrderedDict([('best/input_layer1',
              OrderedDict([('type', 'input_layer'),
                           ('input', []),
                           ('output', ['best/conv1']),
                           ('input_shapes', [[-1, 640, 640, 3]]),
                           ('output_shapes', [[-1, 640, 640, 3]]),
                           ('original_names', ['input']),
                           ('compilation_params', {}),
                           ('q

On the top of the output the output_layers_order is printed. It should look like this. The renamed layers we need to check in de codeblock below and adjust if needed.

```
Inspecting layers from HailoNet (OrderedDict):
Key: name
'best'

================================================================================

Key: net_params
OrderedDict([('version', '1.0'),
             ('stage', 'HN'),
             ('clusters_placement', [[]]),
             ('clusters_to_skip', []),
             ('output_layers_order',
              ['best/conv41',
               'best/conv42',
               'best/conv52',
               'best/conv53',
               'best/conv62',
               'best/conv63']),
             ('transposed_net', False),
             ('net_scopes', ['best'])])
```

Check it the output layers have the correct name in the code in the next codeblock. Adjust if needed. [yolov8s](https://hailo-model-zoo.s3.eu-west-2.amazonaws.com/ObjectDetection/Detection-COCO/yolo/yolov8s/2023-02-02/yolov8s.zip)

In [10]:
import json
import os
from google.colab import drive

# Updated NMS layer configuration dictionary
nms_layer_config = {
    "nms_scores_th": 0.001,
    "nms_iou_th": 0.7,
    "image_dims": [
        640,
        640
    ],
    "max_proposals_per_class": 100,
    "classes": 5,
    "regression_length": 16,
    "background_removal": False,
    "background_removal_index": 0,
    "bbox_decoders": [
        {
            "name": "best/bbox_decoder41",
            "stride": 8,
            "reg_layer": "best/conv41",
            "cls_layer": "best/conv42"
        },
        {
            "name": "best/bbox_decoder52",
            "stride": 16,
            "reg_layer": "best/conv52",
            "cls_layer": "best/conv53"
        },
        {
            "name": "best/bbox_decoder62",
            "stride": 32,
            "reg_layer": "best/conv62",
            "cls_layer": "best/conv63"
        }
    ]
}

# Path to save the updated JSON configuration
output_dir = "/content/"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
output_path = os.path.join(output_dir, "nms_layer_config.json")

# Save the updated configuration as a JSON file
with open(output_path, "w") as json_file:
    json.dump(nms_layer_config, json_file, indent=4)

print(f"NMS layer configuration saved to {output_path}")

NMS layer configuration saved to /content/nms_layer_config.json


Optional: Save nms_layer_config.json to google drive.

In [11]:
from google.colab import drive

# Copy nms_layer_config.json to google drive
!cp /content/nms_layer_config.json /content/gdrive/MyDrive/hailo/nms_layer_config.json

### 2.1 Calibration data


- The dataset should contain at least 1024 representative images (not labeled).
- Use a GPU.

1. Unzip the calibration dataset and rename the folder to `calibrationDataset`.

In [12]:
from google.colab import drive
import os

# Define Paths
calibrationset_path = "/content/gdrive/MyDrive/hailo/calibrationDataset.zip" # @param {type:"string"}
calibrationset_filename = "calibrationDataset" # @param {type:"string"}

try:
  # Unzip the Dataset
  !unzip {calibrationset_path} -d '/content/'

  # Rename the Extracted Folder
  old_path = f'/content/{calibrationset_filename}'
  new_path = '/content/calibrationDataset'
  if os.path.exists(old_path):
    os.rename(old_path, new_path)
  else:
    print(f"Error: {old_path} does not exist.")
except Exception as e:
  print(f"An error occurred: {e}")

Archive:  /content/gdrive/MyDrive/hailo/calibrationDataset.zip
   creating: /content/calibrationDataset/
  inflating: /content/calibrationDataset/train_vzon0003_resized.jpg  
  inflating: /content/calibrationDataset/train_vespsp0184_resized.jpg  
  inflating: /content/calibrationDataset/train_vcra0182_resized.jpg  
  inflating: /content/calibrationDataset/train_vvel0120_resized.jpg  
  inflating: /content/calibrationDataset/train_amel0079_resized.jpg  
  inflating: /content/calibrationDataset/train_vvel0042_resized.jpg  
  inflating: /content/calibrationDataset/train_amel0084_resized.jpg  
  inflating: /content/calibrationDataset/train_vzon0161_resized.jpg  
  inflating: /content/calibrationDataset/train_vespsp0179_resized.jpg  
  inflating: /content/calibrationDataset/train_vzon0073_resized.jpg  
  inflating: /content/calibrationDataset/train_vvel0150_resized.jpg  
  inflating: /content/calibrationDataset/train_amel0009_resized.jpg  
  inflating: /content/calibrationDataset/train_amel

2. Make calibration data. Adjust the size of the image if you input layer has an other format.

In [13]:
import numpy as np
from PIL import Image
import os
from google.colab import drive

# Paths to directories and files
image_dir = '/content/calibrationDataset'
output_dir = '/content/'
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# File paths for saving calibration data
calibration_data_path = os.path.join(output_dir, "calibration_data.npy")
processed_data_path = os.path.join(output_dir, "processed_calibration_data.npy")

# Initialize an empty list for calibration data
calib_data = []

# Process all image files in the directory
for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        img = Image.open(img_path).resize((640, 640))  # CHANGE desired dimensions
        img_array = np.array(img) / 255.0  # Normalize to [0, 1]
        calib_data.append(img_array)

# Convert the calibration data to a NumPy array
calib_data = np.array(calib_data)

# Save the normalized calibration data
np.save(calibration_data_path, calib_data)
print(f"Normalized calibration dataset saved with shape: {calib_data.shape} to {calibration_data_path}")

# Scale the normalized data back to [0, 255]
processed_calibration_data = calib_data * 255.0

# Save the processed calibration data
np.save(processed_data_path, processed_calibration_data)
print(f"Processed calibration dataset saved with shape: {processed_calibration_data.shape} to {processed_data_path}")



Normalized calibration dataset saved with shape: (1024, 640, 640, 3) to /content/calibration_data.npy
Processed calibration dataset saved with shape: (1024, 640, 640, 3) to /content/processed_calibration_data.npy


Restart to clear RAM

In [15]:
%reset -f


Optional: Zip and save both calibration files to google drive. Time to take a coffee...

In [ ]:
from google.colab import drive

# Zip and save processed_calibration_data.npy to google drive hailo folder
!zip -r processedCalibrationData.zip /content/processed_calibration_data.npy

!cp /content/processedCalibrationData.zip /content/gdrive/MyDrive/hailo/

In [ ]:
from google.colab import drive

# Zip and save calibration_data.npy to google drive hailo folder
!zip -r calibrationData.zip  /content/calibration_data.npy

!cp /content/calibrationData.zip /content/gdrive/MyDrive/hailo/

Optional: Remove the folder calibrationDataset.

In [ ]:
import shutil

# Remover calibrationDataset folder
try:
  shutil.rmtree('calibrationDataset')
  print("Folder 'calibrationDataset' removed successfully.")
except FileNotFoundError:
  print("Folder 'calibrationDataset' does not exist.")
except OSError as e:
  print(f"Error removing folder: {e}")

Make alls information

[yolov8s alls](https://github.com/hailo-ai/hailo_model_zoo/blob/master/hailo_model_zoo/cfg/alls/generic/yolov8s.alls)
```
normalization1 = normalization([0.0, 0.0, 0.0], [255.0, 255.0, 255.0])
change_output_activation(conv42, sigmoid)
change_output_activation(conv53, sigmoid)
change_output_activation(conv63, sigmoid)
nms_postprocess("../../postprocess_config/yolov8s_nms_config.json", meta_arch=yolov8l, engine=cpu)
```


In [20]:
with open("optimize_model.py", "w") as f:

    f.write("""

import os
from hailo_sdk_client import ClientRunner

# Define your model's HAR file name
model_name = "best"
hailo_model_har_name = f"{model_name}_hailo_model.har"


# Ensure the HAR file exists
assert os.path.isfile(f"{model_name}_hailo_model.har")

# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=hailo_model_har_name)

# Define the model script to add a normalization layer
# Normalization for [0, 1] range
alls = \"\"\"
quantization_param([conv42, conv53, conv63], force_range_out=[0.0, 1.0])
normalization1 = normalization([0.0, 0.0, 0.0], [255.0, 255.0, 255.0])
change_output_activation(conv42, sigmoid)
change_output_activation(conv53, sigmoid)
change_output_activation(conv63, sigmoid)
nms_postprocess("/content/nms_layer_config.json", meta_arch=yolov8, engine = cpu)
performance_param(compiler_optimization_level=max)
\"\"\"

# Load the model script into the ClientRunner
runner.load_model_script(alls)

# Define a calibration dataset
# Replace 'calib_dataset' with the actual dataset you're using for calibration
# For example, if it's a directory of images, prepare the dataset accordingly
calib_dataset = "/content/processed_calibration_data.npy"

# Perform optimization with the calibration dataset
runner.optimize(calib_dataset)

# Save the optimized model to a new Quantized HAR file
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

print(f"Quantized HAR file saved to: {quantized_model_har_path}")

""")

In [21]:
!my_env/bin/python optimize_model.py

[info] ParsedPerformanceParam command, setting optimization_level(max=2)
[info] Loading model script commands to best from string
[info] ParsedPerformanceParam command, setting optimization_level(max=2)
[info] Starting Model Optimization
[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[warning] Running model optimization with zero level of optimization is not recommended for production use and might lead to suboptimal accuracy results
[info] Model received quantization params from the hn
[info] Starting Mixed Precision
[info] Mixed Precision is done (completion time is 00:00:00.69)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration
Calibration: 100% 64/64 [00:46<00:00,  1.38entries/s]
[info] Statistics Collector is done (completion time is 00:00:49.01)
[warning] The force_range command has been used, notice that its be

Optional: Copy best quantized model.har to google drive.

In [ ]:
!cp /content/best_quantized_model.har /content/gdrive/MyDrive/hailo/best_quantized_model.har

#3. Model compilation

In [ ]:
with open("compile_model.py", "w") as f:

    f.write("""
from hailo_sdk_client import ClientRunner

# Define the quantized model HAR file
model_name = "best"
quantized_model_har_path = f"/content/best_quantized_model.har"

# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=quantized_model_har_path)
print("[info] ClientRunner initialized successfully.")

# Compile the model
try:
    hef = runner.compile()
    print("[info] Compilation completed successfully.")
except Exception as e:
    print(f"[error] Failed to compile the model: {e}")
    raise
file_name = f"{model_name}.hef"
with open(file_name, "wb") as f:
    f.write(hef)
""")

In [ ]:
!my_env/bin/python compile_model.py